# 🧠 Computer Vision Lecture 4.3

## Topic: Live Face Recognition Using Webcam

**Instructor:** Azeem Aslam<br>
**Duration:** 2–3 Hours

## 🎯 What You Will Learn Today

By the end, you will:

* Run real-time face recognition on live webcam feed
* Detect multiple faces per frame
* Recognize known people vs unknown people
* Handle multiple known encodings
* Save live recognition snapshots
* Think like a real CV Engineer

> Today we move from "Face Recognition on Image" → "Face Recognition in Live Video"


# 🔹 Step 0 – Install Required Libraries

If you haven’t installed yet:


In [ ]:
!pip install face-recognition opencv-python matplotlib numpy

- ✅ Works on Windows / Linux / Mac<br>
- ✅ Ensure your webcam is working


# 🔹 Step 1 – Import Libraries


In [2]:
import cv2

In [3]:
import face_recognition

C:\Users\User\anaconda3\Lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [4]:
import numpy as np

In [5]:
import matplotlib.pyplot as plt

In [6]:
import os

# 🔹 Step 2 – Load and Encode Known Faces

In [7]:
known_face_encodings = []
known_face_names = []

In [8]:
# load all images in known_faces folder
try:
    for filename in os.listdir("known_faces"):
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            img_path = os.path.join("known_faces" , filename)
    
            # load image
            img = face_recognition.load_image_file(img_path)
    
            # encode image
            encoding = face_recognition.face_encodings(img)[0]
    
            # Add in list
            known_face_encodings.append(encoding)
            known_face_names.append(os.path.splitext(filename)[0])  # filename without extension

    print("All images Load successfully.")
except Exception as e:
    print("Error in images Load : " , e)


All images Load successfully.


✅ This automatically adds multiple known people.


# 🔹 Step 3 – Start Webcam Feed

In [17]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print(" Webcam not opened. Try changing 0 to 1 or check camera permissions.")
else:
    print("Webcam is open successfully. Press Q to quit")
    process_this_frame = True
    while True:
        ret , frame = cap.read()
        if not ret:
            print("Cannot read frame.Please check it.")
            break

        resize_frame = cv2.resize(frame, (0, 0), fx=0.20, fy=0.20)

        # Convert safely for dlib
        rgb_small_frame = cv2.cvtColor(resize_frame, cv2.COLOR_BGR2RGB)

        if process_this_frame:
                  
            # Detect faces
            face_locations = face_recognition.face_locations(rgb_small_frame , model="hog")
            face_encodings = face_recognition.face_encodings(rgb_small_frame , face_locations)

            names = []

            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(known_face_encodings , face_encoding)
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)

                name = "Unknown"
                best_match_index = np.argmin(face_distances)
                
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]

                names.append(name)

        process_this_frame = not process_this_frame

        # Draw results
        for (top , right , bottom , left) , name in zip(face_locations , names):
            # Scale back coordinates since we resized frame
            scale = 1 / 0.20

            top = int(top * scale)
            right = int(right * scale)
            bottom = int(bottom * scale)
            left = int(left * scale)

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, name, (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)


        cv2.imshow("Live Face Recognition (Press Q to Quit)", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()      
    cv2.destroyAllWindows()       


Webcam is open successfully. Press Q to quit


# 🔹 Step 4 – Optional: Save Snapshots

In [18]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Webcam not opened.")
else:
    print("Webcam is open successfully. Press Q to quit")

    process_this_frame = True
    face_locations = []
    names = []

    out_dir = "recognized_faces"
    os.makedirs(out_dir, exist_ok=True)

    saved_names = set()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        resize_frame = cv2.resize(frame, (0, 0), fx=0.20, fy=0.20)
        rgb_small_frame = cv2.cvtColor(resize_frame, cv2.COLOR_BGR2RGB)

        if process_this_frame:
            face_locations = face_recognition.face_locations(rgb_small_frame, model="hog")
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            names = []

            for face_encoding in face_encodings:
                name = "Unknown"

                # IMPORTANT FIX
                if len(known_face_encodings) > 0:
                    face_distances = face_recognition.face_distance(
                        known_face_encodings, face_encoding
                    )

                    best_match_index = np.argmin(face_distances)

                    if face_distances[best_match_index] < 0.5:
                        name = known_face_names[best_match_index]

                names.append(name)

        process_this_frame = not process_this_frame

        for (top, right, bottom, left), name in zip(face_locations, names):
            scale = 5  # since fx=0.20

            top *= scale
            right *= scale
            bottom *= scale
            left *= scale

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, name, (left, top-10),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

            if name != "Unknown" and name not in saved_names:
                face_img = frame[top:bottom, left:right]
                cv2.imwrite(os.path.join(out_dir, f"{name}.jpg"), face_img)
                saved_names.add(name)

        cv2.imshow("Live Face Recognition", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


Webcam is open successfully. Press Q to quit


# 🔹 Step 5 – Important Notes

* Real-time speed depends on: CPU/GPU + frame resizing
* Resize frame = faster but lower accuracy
* Low light or angled faces → may not recognize
* Multiple faces detected → each compared to all known faces

# ✅ Summary  

* Face Detection = “Where is the face?”
* Face Recognition = “Who is this face?”
* Use webcam for live detection
* Encode known faces → compare with live faces
* Handles multiple known people + unknown
* Snapshot saving = optional, useful for projects
